In [11]:
import pandas as pd
import gdown
import numpy as np
from sklearn.preprocessing import StandardScaler


file_id = "180_b5SgvgfDz1hAsuML2JTgxzQsWenCC"
url = f"https://drive.google.com/uc?id={file_id}"


output = "Housing.csv"
gdown.download(url, output, quiet=False)


data = pd.read_csv(output)


data.head()


Downloading...
From: https://drive.google.com/uc?id=180_b5SgvgfDz1hAsuML2JTgxzQsWenCC
To: /content/Housing.csv
100%|██████████| 30.0k/30.0k [00:00<00:00, 26.4MB/s]


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [2]:
def hypothesis(data, w):


    X = data[['area', 'bedrooms', 'bathrooms']].values


    X = np.hstack((np.ones((X.shape[0], 1)), X))


    y_pred = np.dot(X, w)

    return y_pred

In [3]:
def compute_loss(data, w):

    y_true = data['price'].values


    y_pred = hypothesis(data, w)


    mse = np.mean((y_true - y_pred) ** 2)

    return mse

In [4]:
def gradient_descent_step(data, w, learning_rate=0.01):


    y_true = data['price'].values


    y_pred = hypothesis(data, w)


    X = data[['area', 'bedrooms', 'bathrooms']].values


    X = np.hstack((np.ones((X.shape[0], 1)), X))


    gradient = -(2 / len(y_true)) * np.dot(X.T, (y_true - y_pred))


    w_new = w - learning_rate * gradient

    return w_new


w = np.array([100000, 50, 50000, 20000])
learning_rate = 0.01


w_updated = gradient_descent_step(data, w, learning_rate)


print("Оновлені вагові коефіцієнти:", w_updated)


Оновлені вагові коефіцієнти: [1.84704411e+05 4.74679259e+08 3.10418703e+05 1.38210230e+05]


In [7]:
scaler = StandardScaler()
X = data[['area', 'bedrooms', 'bathrooms']].values
X_scaled = scaler.fit_transform(X)


data_scaled = data.copy()
data_scaled[['area', 'bedrooms', 'bathrooms']] = X_scaled

def gradient_descent(data, w, learning_rate=0.001, num_iterations=10000, tolerance=1e-6):
    prev_loss = float('inf')
    for i in range(num_iterations):
        w = gradient_descent_step(data, w, learning_rate)
        current_loss = compute_loss(data, w)

        # Перевірка на зміну похибки для зупинки
        if np.abs(prev_loss - current_loss) < tolerance:
            print(f"Градієнтний спуск зупинився на {i+1}-й ітерації")
            break
        prev_loss = current_loss

    return w


w = np.random.randn(4)  # Ініціалізація ваг випадковими числами
learning_rate = 0.001  # Зменшена швидкість навчання
num_iterations = 10000  # Максимальна кількість ітерацій


w_best = gradient_descent(data_scaled, w, learning_rate, num_iterations)

print("Найкращі параметри вагового вектора:", w_best)


Найкращі параметри вагового вектора: [4766729.23807627  821214.21860684  299984.14862796  695807.91267749]


In [8]:
def analytical_solution(data):

    X = data[['area', 'bedrooms', 'bathrooms']].values
    X = np.hstack((np.ones((X.shape[0], 1)), X))  # Додаємо стовпець одиниць для перехоплення


    y = data['price'].values


    w_analytical = np.linalg.inv(X.T @ X) @ X.T @ y

    return w_analytical


w_analytical = analytical_solution(data)
print("Вагові коефіцієнти, знайдені аналітично:", w_analytical)


Вагові коефіцієнти, знайдені аналітично: [-1.73171608e+05  3.78762754e+02  4.06820034e+05  1.38604950e+06]


In [13]:
from sklearn.linear_model import LinearRegression

def sklearn_linear_regression(data):

    X = data[['area', 'bedrooms', 'bathrooms']].values


    y = data['price'].values


    model = LinearRegression()


    model.fit(X, y)


    w_sklearn = np.hstack((model.intercept_, model.coef_))

    return w_sklearn, model


w_sklearn, model_sklearn = sklearn_linear_regression(data)

print("Вагові коефіцієнти з sklearn LinearRegression:", w_sklearn)


def predict_with_analytical_solution(data, w):
    X = data[['area', 'bedrooms', 'bathrooms']].values
    X = np.hstack((np.ones((X.shape[0], 1)), X))

    y_pred = X @ w
    return y_pred


y_pred_analytical = predict_with_analytical_solution(data, w_analytical)


X = data[['area', 'bedrooms', 'bathrooms']].values
y_pred_sklearn = model_sklearn.predict(X)


print("Прогнози аналітичного рішення (перші 5):", y_pred_analytical[:5])
print("Прогнози з sklearn LinearRegression (перші 5):", y_pred_sklearn[:5])


Вагові коефіцієнти з sklearn LinearRegression: [-1.73171608e+05  3.78762754e+02  4.06820034e+05  1.38604950e+06]
Прогнози аналітичного рішення (перші 5): [ 7036627.15462757 10392020.79073062  7591864.51496455  7066928.17491437
  5650577.65683657]
Прогнози з sklearn LinearRegression (перші 5): [ 7036627.15462756 10392020.79073062  7591864.51496455  7066928.17491437
  5650577.65683656]
